# Final Project 1: Damped Driven Pendulum

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.pyplot import figure

## Define our coupled derivatives to integrate

In [ ]:
def dydxA(x, f):
    #set the derivatives
    
    y = f[0]
    z = f[1]
    
    #declare an array
    y_derivs = np.zeros_like(f) #this requires _like because of its feature of giving actual values of 0 rather than system os based nulls
    
    #set dydx = z
    y_derivs[0] = z
    
    #set dzdx = -np.sin(y) *** This is part 1  ##############################################################################
    y_derivs[1] = -np.sin(y)
    
    #here we have to return the array
    return y_derivs

In [ ]:
def dydxB(x, f):
    #set the derivatives
    
    y = f[0]
    z = f[1]
    
    #declare an array
    y_derivs = np.zeros_like(f) #this requires _like because of its feature of giving actual values of 0 rather than system os based nulls
    
    #set dydx = z
    y_derivs[0] = z
    
    #set dzdx 
    y_derivs[1] = -np.sin(y)-(z/2)
    
    #here we have to return the array
    return y_derivs

In [ ]:
def dydxC(x, f):
    #set the derivatives
    
    y = f[0]
    z = f[1]
    
    #declare an array
    y_derivs = np.zeros_like(f) #this requires _like because of its feature of giving actual values of 0 rather than system os based nulls
    
    #set dydx = z
    y_derivs[0] = z
    
    #set dzdx
    y_derivs[1] = -np.sin(y)-(z/2)+(0.5)*(np.sin(x*(2/3)))
    
    #here we have to return the array
    return y_derivs

In [ ]:
def dydxD(x, f):
    #set the derivatives
    
    y = f[0]
    z = f[1]
    
    #declare an array
    y_derivs = np.zeros_like(f) #this requires _like because of its feature of giving actual values of 0 rather than system os based nulls
    
    #set dydx = z
    y_derivs[0] = z
    
    #set dzdx
    y_derivs[1] = -np.sin(y)-(z/2)+(1.46)*(np.sin(x*(2/3)))
    
    #here we have to return the array
    return y_derivs

## Define the Cash-Karp core function

In [ ]:
def cash_karp_core_mv(dydx, xi, yi, nv, h, case):
    
    #declare k# arrays (https://www.researchgate.net/profile/Proceso-Fernandez/publication/316007527/figure/fig1/AS:482085128413184@1491949459014/Butcher-Tableau-for-Cash-Karp.png)
    kr = 7 #Row of 7
    kc = 6 #Column of 6
    
    #declare the array for each section (Left of the vertical line, centre stage, or below the horizontal line)
    l = np.zeros(kr)
    c1 = np.zeros((kr, kc))
    b1 = np.zeros(kr)
    b2 = np.zeros(kr)
    
    #First part - Left of the vertical line (l) (0, 1/5, 3/10, 3/5, 1, 7/8)
    l[1] = 1./5. #l[0] is ignored because it is already 0
    l[2] = 3./10.
    l[3] = 3./5.
    l[4] = 1.
    l[5] = 7./8.
    
    #Second part - Centre stage:
    
        #Column 1 (0, 1/5, 3/40, 3/10, -11/54, 1631/55296):
    c1[2,1] = 1./5. #[0, 1; #] are ignored because 1 is 0 and we will iterate according to non-array rules i.e. 1-># not 0->#
    c1[3,1] = 3./40.
    c1[4,1] = 3./10.
    c1[5,1] = -11./54.
    c1[6,1] = 1631./55296.
    
        #Column 2 (0, 9/40, -9/10, 5/2, 175/512):
    c1[3,2] = 9./40.
    c1[4,2] = -9./10.
    c1[5,2] = 5./2.
    c1[6,2] = 175./512.
    
        #Column 3 (0, -70/27, 575/13824):
    c1[4,3] = 6/5
    c1[5,3] = -70/27
    c1[6,3] = 575./13824.

        #Column 4 (0, 35/27, 44275/110592):
    c1[5,4] = 35./27.
    c1[6,4] = 44275./110592.
    
        #Column FINAL (0, 253/4096):
    c1[6,5] = 253./4096.
    
    ## The final column, just as the others, is already set to 0 ##
    
    #Third part - Lower stage:
    
        #Row 1 (37/378, 0, 250/621, 125/594, 0, 512/1771): -- This is the 4th order solution values
    b1[1] = 37./378.
    b1[2] = 0.
    b1[3] = 250./621.
    b1[4] = 125./594.
    b1[5] = 0.
    b1[6] = 512./1771.
    
        #Row 1 (2825/27648, 0, 18575/48384, 13525/55296, 277/14336, 1/4): -- This is the 5th order solution values
    b2[1] = 2825./27648.
    b2[2] = 0.
    b2[3] = 18575./48384.
    b2[4] = 13525./55296.
    b2[5] = 277./14336.
    b2[6] = 1./4.
    
    
    #define the k array
    ki = np.zeros((kr,nv)) #Create an 2D array for values along the Butcher Tableau for the Cash-Karp method along the length of the function
    
    while(case == "A"):
        #compute ki
        for i in range(1, kr): #Iterating through each column or the value of each row depending on the given column
            xn = xi + l[i]*h #****Iterate through values of x such that it corresponds with the given array value on the Left hand side of the verticle line

            yn = yi.copy() #****y values need to be reset each loop
            for j in range(1, i):
                yn += c1[i, j]*ki[j,:] #This line tells yn to add to itself (set before to be the initial y value given) the array value for each column

            ki[i,:] = h*dydxA(xn, yn) #we are multiplying the function dydx (done previously) by the initial step value "h"
        case = "done"
            
    while(case == "B"):
        #compute ki
        for i in range(1, kr): #Iterating through each column or the value of each row depending on the given column
            xn = xi + l[i]*h #****Iterate through values of x such that it corresponds with the given array value on the Left hand side of the verticle line

            yn = yi.copy() #****y values need to be reset each loop
            for j in range(1, i):
                yn += c1[i, j]*ki[j,:] #This line tells yn to add to itself (set before to be the initial y value given) the array value for each column

            ki[i,:] = h*dydxB(xn, yn) #we are multiplying the function dydx (done previously) by the initial step value "h"
        case = "done"
            
    while(case == "C"):
        #compute ki
        for i in range(1, kr): #Iterating through each column or the value of each row depending on the given column
            xn = xi + l[i]*h #****Iterate through values of x such that it corresponds with the given array value on the Left hand side of the verticle line

            yn = yi.copy() #****y values need to be reset each loop
            for j in range(1, i):
                yn += c1[i, j]*ki[j,:] #This line tells yn to add to itself (set before to be the initial y value given) the array value for each column

            ki[i,:] = h*dydxC(xn, yn) #we are multiplying the function dydx (done previously) by the initial step value "h"
        case = "done"
            
            
    while(case == "D"):
        #compute ki
        for i in range(1, kr): #Iterating through each column or the value of each row depending on the given column
            xn = xi + l[i]*h #****Iterate through values of x such that it corresponds with the given array value on the Left hand side of the verticle line

            yn = yi.copy() #****y values need to be reset each loop
            for j in range(1, i):
                yn += c1[i, j]*ki[j,:] #This line tells yn to add to itself (set before to be the initial y value given) the array value for each column

            ki[i,:] = h*dydxD(xn, yn) #we are multiplying the function dydx (done previously) by the initial step value "h"
        case = "done"
        
            
    #
    ynpo = yi.copy()
    ynpos = yi.copy()
    
    for i in range(1, kr): #this section iterates through the rows and gets the difference of the fourth and firth order solutions for Cash-Karp
        ynpo += b1[i] * ki[i,:] #The first is the fourth order and the second is the fifth order
        ynpos += b2[i] * ki[i,:] #in this section I used trial and error to find that the lower section gave values of are fourth and fifth order solutions, I got these from https://en.wikipedia.org/wiki/Cash%E2%80%93Karp_method
        #These are needed to get an error value "Delta"
        
    #getting the error
    Delta = np.fabs(ynpo-ynpos)
    
    return ynpo, Delta

## Define an adaptive step size drive for Cash-Karp

In [ ]:
def cash_karp_mv_ad(dydx, xi, yi, nv, h, tol, case): #This code is pretty much all from the RK MV Method
    
    #define safety scale
    SAFETY    = 0.9
    H_NEW_FAC = 2.0
    
    #set a maximum number of iterations
    imax = 1000
    
    #set an iteration variable
    i = 0
    
    #create an error
    Delta = np.full(nv, 2*tol)
    
    #remember the step
    h_step = h
    
    #adjust step
    while(Delta.max()/tol > 1.0):
        #We need to get the return values of the function cash_karp_core_mv()
        yipo, Delta = cash_karp_core_mv(dydx, xi, yi, nv, h_step, case) #Delta is the main value that we want, because that's what defines the new step value
        
        #if the error is too large, take a smaller step
        if(Delta.max()/tol > 1.0):
            
            #our error is too large, decrease the step
            h_step *= SAFETY * (Delta.max()/tol)**(-0.25)
            
        #check the iteration
        if (i >= imax):
            print("Too many iterations in cash_karp_mv_ad()")
            raise StopIteration("Ending after i = ", i)
            
        #iterate
        i += 1
        
    #next time we should try to make a bigger step
    h_new = np.fmin(h_step * (Delta.max()/tol)**(-0.9), h_step*H_NEW_FAC)
    
    #return the answer, a new step, and the step we actually took
    return yipo, h_new, h_step

## Define a wrapper for Cash-Karp

In [ ]:
def cash_karp_mv(dfdx, a, b, y_a, tol, case): #Most of this code was from the RK mv program as well
    
    #dfdx is the derivative with respect to x
    #a is the lower bound
    #b is the upper bound
    #y_a are the boundary conditions
    #tol is the tolerance for integrating y
    
    #define our starting step
    xi = a
    yi = y_a.copy()
    
    #an initial step size == make very teeny weeny!
    h = 1.0e-4 * (b-a)
    
    #set a maximum number of iterations
    imax = 1000000 #I know this seems absurd, but it is quite necessary
    
    #set an iteration variable
    i = 0
    
    #set the number of coupled odes to the size of y_a
    nv = len(y_a)
    
    #set the intial conditions
    x = np.full(1, a)
    y = np.full((1, nv), y_a)
    
    #set a flag
    flag = 1
    
    #loop until we reach the right side
    while(flag):
        
        #calculate y_i+1
        yi_new, h_new, h_step = cash_karp_mv_ad(dydx, xi, yi, nv, h, tol, case)
        
        #update the step
        h = h_new
        
        #prevent an overshoot
        if(xi + h_step > b):
            #take a smaller step
            h = b - xi
            
            #recalculate y_i + 1
            yi_new, h_new, h_step = cash_karp_mv_ad(dydx, xi, yi, nv, h, tol, case) #This one here uses the new value of "h" (the step value)
            
            #break
            flag = 0
            
        #update values
        xi += h_step
        yi[:] = yi_new.copy()
        
        #add the step to the arrays
        x = np.append(x, xi)
        yi_new = np.zeros((len(x), nv))
        yi_new[0:len(x) - 1, :] = y[:]
        yi_new[-1, :] = yi[:]
        
        del y
        y = yi_new
        
        #prevent too many iterations
        if(i >= imax):
            print("Maximum iterations reached.")
            raise StopIteration("Iteration number = ", i)
            
        #iterate
        i += 1
        
        #break if new xi is == b
        if(xi == b):
            flag = 0
            
    #return the answer
    return x,y

## Perform the integration

In [ ]:
### Case A ###

a = -1.0    #*
b = 100     #*

y_0 = np.zeros(2)
x_0 = np.zeros(2)
y_0[0] = 0.0
y_0[1] = 1.0
nv = 2

case = "A"

tolerance = 1.0e-6

i = 0

#perform the integration
x,y = cash_karp_mv(dydx, a, b, y_0, tolerance, case)
figure(figsize=(15, 15), dpi=100)

y_new = y.copy()

#This is a very rough and probably poor approximation of theta
if case != "B":
    for i in range(len(y)):
        y_new[i] = y[i-100] - y[i] #Ok so this is a horrible method of integration that does not properly account for any errors, but it was the best method with what I could perform
else:
    y_new[i] = y[i-20]-y[i]

y_new[0] = 3.14    

plt.subplot(3, 2, 1)
plt.plot(x, y_new[:, 1], 'o', label='Θ(t)')
plt.title("Θ Vs. t (Case A)")
plt.xlabel('t')
plt.ylabel('Θ(t)')
plt.xlim([0, 100])
plt.ylim([-20, 20])
plt.grid(color='b', linestyle='--', linewidth=0.2)
plt.legend(frameon=False)

plt.subplot(3, 2, 2)
plt.title("dΘ/dt Vs. t (Case A)")
plt.xlabel('t')
plt.ylabel('dΘ/dt')
plt.plot(x, y[:,1], 'o', label='dΘ/dt')
plt.grid(color='b', linestyle='--', linewidth=0.2)
plt.xlim([0, 100])
plt.ylim([-20, 20])

plt.subplot(3, 2, 3)
plt.plot(y[:,1], y_new[:, 1], 'o', label='dΘ/dt')
plt.title("dΘ/dt Vs. Θ (Case A)")
plt.xlabel('Θ')
plt.ylabel('dΘ/dt')
plt.grid(color='b', linestyle='--', linewidth=0.2)
plt.legend(frameon=False)

plt.tight_layout()
plt.legend(frameon=False)

In [ ]:
### Case B ###

a = -1.0    #*
b = 100     #*

y_0 = np.zeros(2)
x_0 = np.zeros(2)
y_0[0] = 0.0
y_0[1] = 1.0
nv = 2

case = "B"

tolerance = 1.0e-6

i = 0

#perform the integration
x,y = cash_karp_mv(dydx, a, b, y_0, tolerance, case)
figure(figsize=(15, 15), dpi=100)

y_new = y.copy()

#This is a very rough and probably poor approximation of theta
if case != "B":
    for i in range(len(y)):
        y_new[i] = y[i-100] - y[i]
else:
    y_new[i] = y[i-20]-y[i]

y_new[0] = 3.14    

plt.subplot(3, 2, 1)
plt.plot(x, y_new[:, 1], 'o', label='Θ(t)')
plt.title("Θ Vs. t (Case B)")
plt.xlabel('t')
plt.ylabel('Θ(t)')
plt.xlim([0, 100])
plt.ylim([-20, 20])
plt.grid(color='b', linestyle='--', linewidth=0.2)
plt.legend(frameon=False)

plt.subplot(3, 2, 2)
plt.title("dΘ/dt Vs. t (Case B)")
plt.xlabel('t')
plt.ylabel('dΘ/dt')
plt.plot(x, y[:,1], 'o', label='dΘ/dt')
plt.grid(color='b', linestyle='--', linewidth=0.2)
plt.xlim([0, 100])
plt.ylim([-20, 20])

plt.subplot(3, 2, 3)
plt.plot(y[:,1], y_new[:, 1], 'o', label='dΘ/dt')
plt.title("dΘ/dt Vs. Θ (Case B)")
plt.xlabel('Θ')
plt.ylabel('dΘ/dt')
plt.grid(color='b', linestyle='--', linewidth=0.2)
plt.legend(frameon=False)

plt.tight_layout()
plt.legend(frameon=False)

In [ ]:
### Case C ###

a = -1.0    #*
b = 100     #*

y_0 = np.zeros(2)
x_0 = np.zeros(2)
y_0[0] = 0.0
y_0[1] = 1.0
nv = 2

case = "C"

tolerance = 1.0e-6

i = 0

#perform the integration
x,y = cash_karp_mv(dydx, a, b, y_0, tolerance, case)
figure(figsize=(15, 15), dpi=100)

y_new = y.copy()

#This is a very rough and probably poor approximation of theta
if case != "B":
    for i in range(len(y)):
        y_new[i] = y[i-100] - y[i]
else:
    y_new[i] = y[i-20]-y[i]

y_new[0] = 3.14    


plt.subplot(3, 2, 1)
plt.plot(x, y_new[:, 1], 'o', label='Θ(t)')
plt.title("Θ Vs. t (Case C)")
plt.xlabel('t')
plt.ylabel('Θ(t)')
plt.xlim([0, 100])
plt.ylim([-20, 20])
plt.grid(color='b', linestyle='--', linewidth=0.2)
plt.legend(frameon=False)

plt.subplot(3, 2, 2)
plt.title("dΘ/dt Vs. t (Case C)")
plt.xlabel('t')
plt.ylabel('dΘ/dt')
plt.plot(x, y[:,1], 'o', label='dΘ/dt')
plt.grid(color='b', linestyle='--', linewidth=0.2)
plt.xlim([0, 100])
plt.ylim([-20, 20])

plt.subplot(3, 2, 3)
plt.plot(y[:,1], y_new[:, 1], 'o', label='dΘ/dt')
plt.title("dΘ/dt Vs. Θ (Case C)")
plt.xlabel('Θ')
plt.ylabel('dΘ/dt')
plt.grid(color='b', linestyle='--', linewidth=0.2)
plt.legend(frameon=False)

plt.tight_layout()
plt.legend(frameon=False)

In [ ]:
### Case D ###

a = -1.0    #*
b = 100     #*

y_0 = np.zeros(2)
x_0 = np.zeros(2)
y_0[0] = 0.0
y_0[1] = 1.0
nv = 2

case = "D"

tolerance = 1.0e-6

i = 0

#perform the integration
x,y = cash_karp_mv(dydx, a, b, y_0, tolerance, case)
figure(figsize=(15, 15), dpi=100)

y_new = y.copy()

#This is a very rough and probably poor approximation of theta
if case != "B":
    for i in range(len(y)):
        y_new[i] = y[i-100] - y[i]
else:
    y_new[i] = y[i-20]-y[i]

y_new[0] = 3.14    

#Subplots are not fun ;-;

plt.subplot(3, 2, 1)
plt.plot(x, y_new[:, 1], 'o', label='Θ(t)')
plt.title("Θ Vs. t (Case D)")
plt.xlabel('t')
plt.ylabel('Θ(t)')
plt.xlim([0, 100])
plt.ylim([-20, 20])
plt.grid(color='b', linestyle='--', linewidth=0.2)
plt.legend(frameon=False)

plt.subplot(3, 2, 2)
plt.title("dΘ/dt Vs. t (Case D)")
plt.xlabel('t')
plt.ylabel('dΘ/dt')
plt.plot(x, y[:,1], 'o', label='dΘ/dt')
plt.grid(color='b', linestyle='--', linewidth=0.2)
plt.xlim([0, 100])
plt.ylim([-20, 20])

plt.subplot(3, 2, 3)
plt.plot(y[:,1], y_new[:, 1], 'o', label='dΘ/dt')
plt.title("dΘ/dt Vs. Θ (Case D)")
plt.xlabel('Θ')
plt.ylabel('dΘ/dt')
plt.grid(color='b', linestyle='--', linewidth=0.2)
plt.legend(frameon=False)

plt.tight_layout()
plt.legend(frameon=False)